# Jedha Kayak project #block1
Notebook for development

In [30]:
# import
import os
import uuid
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dotenv import load_dotenv
from IPython.display import display
from geopy.geocoders import Nominatim

In [16]:
# env variables
load_dotenv()
OPEN_WEATHER_TOKEN = os.getenv('Open_weather_token')

## 1. Get weather data

In [17]:
french_top_35 = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre",
                "Rouen", "Paris", "Amiens", "Lille", "Strasbourg",
                "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim",
                "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
                "Gorges du Verdon", "Bormes les Mimosas", "Cassis",
                "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes",
                "Aigues Mortes", "Saintes Maries de la mer", "Collioure",
                "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz",
                "Bayonne", "La Rochelle"]

In [18]:
# init dataframe
weather_df = pd.DataFrame(french_top_35, columns=["cities"])
display(weather_df.sample(2))

,cities
18,Bormes les Mimosas
31,Montauban


In [19]:
# coordinates of cities
def cities_to_coords(city_name) :
    """ get cities coordiniates.
    """

    # geocoder
    geolocator = Nominatim(user_agent="app")
    location = geolocator.geocode(city_name)

    # full_address, latitude, longitude
    full_address = location.address
    latitude = location.latitude
    longitude = location.longitude

    # return 
    return pd.Series([full_address, latitude, longitude])

## RUN
if not "full_address" in weather_df.columns :
    weather_df[["full_address", "latitude", "longitude"]] = weather_df["cities"].apply(cities_to_coords)
    display(weather_df.sample(2))

else : 
    display(weather_df.sample(2))
    print("full_address, latitude & longitude columns exists !")

,cities,full_address,latitude,longitude
2,Bayeux,"Bayeux, Calvados, Normandie, France métropolit...",49.276462,-0.702474
16,Lyon,"Lyon, Métropole de Lyon, Circonscription dépar...",45.757814,4.832011


In [20]:
def volume_rain(lat, lon, exclude, appid) :
    """ volume of rain in each city over 7 next days.
    """

    # open weather one call api
    params = (
        ('lat', lat),
        ('lon', lon),
        ('exclude', exclude),
        ('appid', appid)
    )

    response = requests.get('https://api.openweathermap.org/data/2.5/onecall', params=params)

    # normalize reponse
    open_weather_7days = pd.json_normalize(response.json()["daily"])

    # clean rain column
    open_weather_7days["rain"] = open_weather_7days["rain"].apply(lambda x : 0 if np.isnan(x) else x) 

    # calculate volumn 
    n_days = open_weather_7days.shape[0]
    volume_rain_7days =  np.sum(open_weather_7days["rain"] * open_weather_7days["pop"]) / n_days

    # return 
    return round(volume_rain_7days,3)

In [21]:
# fill weather_df
exclude = 'current,minutely,hourly,alerts'
appid = OPEN_WEATHER_TOKEN
if not "volume_rain_7days" in weather_df.columns :
    weather_df["volume_rain_7days"] = weather_df[["latitude", "longitude"]].transpose().apply(lambda x : volume_rain(x[0], x[1], exclude, appid))

display(weather_df.sample(2))

,cities,full_address,latitude,longitude,volume_rain_7days
2,Bayeux,"Bayeux, Calvados, Normandie, France métropolit...",49.276462,-0.702474,7.087
17,Gorges du Verdon,"Gorges du Verdon, Route des Crêtes, Les Ferrai...",43.749656,6.328562,0.319


In [22]:
# unique identifier (uuid)
if not "uuid" in weather_df.columns :
    weather_df['uuid'] = weather_df.index.to_series().map(lambda x: uuid.uuid4())

display(weather_df.sample(2))

,cities,full_address,latitude,longitude,volume_rain_7days,uuid
17,Gorges du Verdon,"Gorges du Verdon, Route des Crêtes, Les Ferrai...",43.749656,6.328562,0.319,0af6c004-8219-4a45-8246-9f6f4f7dfd04
23,Uzes,"Uzès, Nîmes, Gard, Occitanie, France métropoli...",44.012128,4.419672,0.236,1cf1b0a6-b1e7-4de3-9789-11b27fe43139


In [23]:
# reorder columns
keep_col = ['uuid', 'cities', 'full_address', 'latitude', 'longitude', 'volume_rain_7days']
weather_df = weather_df[keep_col]

display(weather_df.sample(2))

,uuid,cities,full_address,latitude,longitude,volume_rain_7days
15,a3db85c7-87ee-497f-94a7-da661d756644,Grenoble,"Grenoble, Isère, Auvergne-Rhône-Alpes, France ...",45.187560,5.735782,6.612
34,b5d388c7-5b63-41c9-92fb-db03d9c5b47e,La Rochelle,"La Rochelle, Charente-Maritime, Nouvelle-Aquit...",46.159113,-1.152043,4.782


In [24]:
# save dataframe
weather_data = "../data/temp/weather_data.csv"
overwrite = False 

if not os.path.exists(weather_data) or overwrite==True:
    weather_df.to_csv(weather_data)

else :
    print(f"{weather_data} exists !")

../data/temp/weather_data.csv exists !


In [25]:
# fetch best destinations
top_5_destinations = weather_df.sort_values(by=['volume_rain_7days'])[:5]
display(top_5_destinations)


,uuid,cities,full_address,latitude,longitude,volume_rain_7days
18,650e2fdf-1348-482d-9427-66d9a6dc6001,Bormes les Mimosas,"Bormes-les-Mimosas, Toulon, Var, Provence-Alpe...",43.157217,6.329254,0.158
23,1cf1b0a6-b1e7-4de3-9789-11b27fe43139,Uzes,"Uzès, Nîmes, Gard, Occitanie, France métropoli...",44.012128,4.419672,0.236
24,8930da08-62dc-4608-a65f-5bf3df2c0e81,Nimes,"Nîmes, Gard, Occitanie, France métropolitaine,...",43.837425,4.360069,0.261
17,0af6c004-8219-4a45-8246-9f6f4f7dfd04,Gorges du Verdon,"Gorges du Verdon, Route des Crêtes, Les Ferrai...",43.749656,6.328562,0.319
22,6bd0796c-725d-4915-9e13-5b2c69d28a39,Avignon,"Avignon, Vaucluse, Provence-Alpes-Côte d'Azur,...",43.949249,4.805901,0.384


In [51]:
# plot
df = top_5_destinations.copy()
df = df.rename(columns={"volume_rain_7days": "Volume of rain"})
# mapbox token
#mapbox_token = "pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w"

# fig
px.set_mapbox_access_token(open("../.mapbox_token").read())
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude",     
                        color="Volume of rain", size="Volume of rain",
                        color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=15, zoom=5)

fig.show()